
# 🧠 Cyber-Fractal-Entanglement: Full Pipeline Documentation

This notebook serves as the **complete documentation and executable demo** for the cyber-fractal-entanglement system, a proof-of-concept that tests whether **Quantum Mutual Information (QMI)** between fractal fingerprints of financial assets can provide **early-warning signals** for equity drawdowns.

---

## 🎯 Project Objective

We aim to evaluate whether the **systemic entanglement signal** \( C(t) \), computed via QMI between asset-level fractal dimensions, statistically **predicts drawdowns in the S&P 500 (SPX)** over short horizons.

---



## 🧩 Fractal Dimension Estimation

We compute time-varying fractal dimensions \( D_i(t) \) for each asset \( i \in \{\text{SPX}, \text{BTC}, \text{GOLD}, \dots\} \) using **Higuchi's method** and compare it with **Hurst Exponent**.

### Why Higuchi?

| Criterion              | Higuchi                         | Hurst                             |
|------------------------|----------------------------------|-----------------------------------|
| Noise Robustness       | ✅ Robust on short/noisy data    | ❌ Sensitive to non-stationarity  |
| Window Size            | ✅ Works with 30–50 pts          | ❌ Needs 100+ pts                 |
| Interpretability       | ✅ Fractal scale [1,2]           | ✅ Persistence scale [0,1]        |
| Literature Support     | ✅ Strong in econophysics        | ⚠️ Classic, less fractal-focused  |

### Higuchi Equation (Simplified)

For time series \( X = \{x_1, x_2, \dots, x_N\} \), the Higuchi fractal dimension is computed from the slope of:

\[
L(k) \sim k^{-D}
\]

Where \( L(k) \) is the average curve length at scale \( k \), and \( D \) is the fractal dimension.


In [ ]:

# Compute fractal dimensions using Higuchi
!python src/fractal.py -i data/processed/prices.csv -w 30



---

## 🔗 Quantum Mutual Information Pipeline

Given fractal time series \( D_i(t) \), we compute:

1. Covariance matrix \( \Sigma_t \) over a rolling window
2. Normalize to density matrix:
\[
\rho_t = \frac{\Sigma_t}{\mathrm{Tr}(\Sigma_t)}
\]
3. Von Neumann entropy for marginals:
\[
S(\rho_i) = -\mathrm{Tr}(\rho_i \log \rho_i)
\]
4. Joint entropy for pair \( (i,j) \) and compute **Quantum Mutual Information**:
\[
\text{QMI}_{ij}(t) = S(\rho_i) + S(\rho_j) - S(\rho_{ij})
\]
5. Aggregate:
\[
C(t) = \sum_{i<j} \text{QMI}_{ij}(t)
\]

---


In [ ]:

# Compute QMI signal C(t)
!python src/qmi.py -i data/processed/fractal_series.csv -o data/processed/C_series.csv



## ✅ Predictive Validation

We define a binary label \( y_t \) indicating a drawdown in SPX over horizon \( H \):

\[
y_t = \mathbb{1}\left[ \exists k \in [1,H]: \frac{P_{t+k} - \max_{u \le t} P_u}{\max_{u \le t} P_u} \leq \theta \right]
\]

### Metrics Used

- **Granger causality** \( p \)-value for \( C(t) \to y_t \)
- **Logistic Regression** significance + AUC
- **ROC AUC** on held-out samples


In [ ]:

# Validate predictive performance
!python src/validate_ct.py --c_series data/processed/C_series.csv --threshold -0.02 --horizon 5



---

## 📈 QMI Surface Visualization

The following figure displays the QMI surface \( \text{QMI}_{ij}(t) \) across time and asset pairs.

- Bright peaks = strong entanglement (high mutual info)
- Dark troughs = decoupling or noise



In [ ]:

from IPython.display import Image
Image(filename='figures/qmi_evolution.png', width=900)



---

## 🧠 Summary

- Higuchi-FD gives robust asset fingerprints
- QMI reveals dynamic system-level entanglement
- Signal \( C(t) \) can offer **statistically significant early-warning signals**
- Grid search shows AUC > 0.6 and \( p < 0.05 \) in select SPX scenarios

---

For more details, see:
- `src/fractal.py`: FD computation
- `src/qmi.py`: QMI and \( C(t) \) generation
- `src/validate_ct.py`: predictive model validation
